In [ ]:
import os
os.environ['R_HOME'] = r'C:\Program Files\R\R-4.4.2'  # Ajuste o caminho conforme sua instalação
os.environ['R_USER'] = r'C:\Program Files\R\R-4.4.2'  # Opcional

In [ ]:
from rpy2.rinterface_lib.callbacks import logger as rpy2_logger
from rpy2.robjects import r, pandas2ri
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import root_mean_squared_error
from statsmodels.stats.diagnostic import het_breuschpagan, het_white, acorr_breusch_godfrey, het_arch, acorr_ljungbox
from statsmodels.stats.stattools import durbin_watson
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf, adfuller
import rpy2.robjects as ro
from scipy.stats import ttest_rel

In [ ]:
# Desabilitar logs do R para evitar mensagens de erro no console
rpy2_logger.setLevel("ERROR")

In [ ]:
%load_ext rpy2.ipython

In [ ]:
tickers=['VALE3.SA', 
'PETR4.SA', 
'ITUB4.SA', 
'PETR3.SA', 
'BBDC4.SA', 
'BBAS3.SA', 
'ELET3.SA', 
'B3SA3.SA', 
'WEGE3.SA', 
'SBSP3.SA', 
'ITSA4.SA', 
'ABEV3.SA', 
'BPAC11.SA', 
'EQTL3.SA', 
'RENT3.SA', 
'JBSS3.SA', 
'PRIO3.SA', 
'RADL3.SA', 
'SUZB3.SA', 
'EMBR3.SA', 
'RAIL3.SA', 
'VBBR3.SA', 
'BBSE3.SA', 
'GGBR4.SA', 
'CMIG4.SA', 
'BRFS3.SA', 
'VIVT3.SA', 
'BBDC3.SA', 
'ENEV3.SA', 
'HAPV3.SA', 
'CPLE6.SA', 
'KLBN11.SA', 
'LREN3.SA', 
'TOTS3.SA', 
'CSAN3.SA', 
'ENGI11.SA', 
'TIMS3.SA', 
'CCRO3.SA', 
'ELET6.SA', 
'HYPE3.SA', 
'EGIE3.SA', 
'SANB11.SA', 
'STBP3.SA', 
'TRPL4.SA', 
'CSNA3.SA', 
'MULT3.SA', 
'TAEE11.SA', 
'FLRY3.SA', 
'GOAU4.SA', 
'CPFE3.SA', 
'AZZA3.SA', 
'CYRE3.SA', 
'BRAP4.SA', 
'BRKM5.SA', 
'CRFB3.SA', 
'MRFG3.SA', 
'MGLU3.SA', 
'IRBR3.SA', 
'SMTO3.SA', 
'SLCE3.SA', 
'USIM5.SA', 
'YDUQ3.SA', 
'MRVE3.SA', 
'COGN3.SA', 
'BEEF3.SA', 
'AZUL4.SA', 
'EZTC3.SA', 
'ALPA4.SA', 
'CVCB3.SA',]

In [ ]:
cdi_data_train = pd.read_csv('SELICTRAIN.csv',sep=';', parse_dates=['Date'],dayfirst=True)
cdi_data_train.set_index('Date', inplace=True)
cdi_data_train.index = pd.to_datetime(cdi_data_train.index, dayfirst=True)
cdi_data_train['CDI_Return'] = cdi_data_train['return'].str.replace(',', '.').astype(float)
cdi_data_train.drop(columns=['return'], inplace=True)
#cdi_data_train['CDI_Return']=np.log(1+cdi_data_train['CDI_Return'])


cdi_data_forecast = pd.read_csv('SELICFORECAST.csv',sep=';', parse_dates=['Date'],dayfirst=True)
cdi_data_forecast.set_index('Date', inplace=True)
cdi_data_forecast.index = pd.to_datetime(cdi_data_forecast.index, dayfirst=True)
cdi_data_forecast['CDI_Return'] = cdi_data_forecast['return'].str.replace(',', '.').astype(float)
cdi_data_forecast.drop(columns=['return'], inplace=True)
#cdi_data_forecast['CDI_Return']=np.log(1+cdi_data_forecast['CDI_Return'])

In [ ]:
start_train = '2019-10-01'
end_train = '2023-04-10'
start_forecasting = '2023-04-06'
end_forecasting = '2024-10-03'

In [ ]:
market_train = yf.download('^BVSP', start=start_train, end=end_train)['Adj Close'].pct_change().dropna()
#market_train= np.log(market_train).diff().dropna()
market_forecast = yf.download('^BVSP', start=start_forecasting, end=end_forecasting)['Adj Close'].pct_change().dropna()
#market_forecast= np.log(market_forecast).diff().dropna()

In [ ]:
cdi_data_train = cdi_data_train.loc[market_train.index]
cdi_data_forecast = cdi_data_forecast.loc[market_forecast.index]

In [ ]:
market_excess_train=(market_train-cdi_data_train['CDI_Return'])*100
market_excess_forecast=(market_forecast-cdi_data_forecast['CDI_Return'])*100

In [ ]:
stock_excess_train=[]
stock_excess_forecast=[]

In [ ]:
for ticker in tickers:
    stock_train=yf.download(ticker, start=start_train, end=end_train)['Adj Close'].pct_change().dropna()#.pct_change()
    #stock_train=np.log(stock_train).diff().dropna()

    
    data= pd.DataFrame({
        'Ticker': ticker,
        'Stock_Return': stock_train,
        'CDI_Return': cdi_data_train['CDI_Return']
    }).dropna()

    data['Stock_Excess'] = ((data['Stock_Return'] - data['CDI_Return']))*100

    stock_excess_train.append(data)

    #forecast
    stock_forecast=yf.download(ticker, start=start_forecasting, end=end_forecasting)['Adj Close'].pct_change().dropna()
    #stock_forecast=np.log(stock_forecast).diff().dropna()

    
    data_forecast= pd.DataFrame({
        'Ticker': ticker,
        'Stock_Return': stock_forecast,
        'CDI_Return': cdi_data_forecast['CDI_Return']
    }).dropna()

    data_forecast['Stock_Excess'] = ((data_forecast['Stock_Return'] - data_forecast['CDI_Return']))*100

    stock_excess_forecast.append(data_forecast)

In [ ]:
stock_excess_train = pd.concat(stock_excess_train)
stock_excess_train = stock_excess_train[['Ticker', 'Stock_Excess']]
stock_excess_forecast = pd.concat(stock_excess_forecast)
stock_excess_forecast = stock_excess_forecast[['Ticker', 'Stock_Excess']]

In [ ]:
# Verificar valores ausentes nos dados
for ticker in tickers:
    stock_train = yf.download(ticker, start=start_train, end=end_train)['Adj Close']
    if stock_train.isnull().any():
        print(f"Ticker {ticker} contém valores ausentes.")
    else:
        print(f"Ticker {ticker} está limpo.")

In [ ]:
tests_capm=[]
params_capm=[]

for ticker in stock_excess_train['Ticker'].unique():
    ticker_excess_train = stock_excess_train.loc[stock_excess_train['Ticker'] == ticker]['Stock_Excess']
    market_excess = market_excess_train.loc[ticker_excess_train.index]
    
    # Ajusta o modelo CAPM Clássico
    #adicionar lags no x e no y
    X_capm = sm.add_constant(market_excess_train.rename('Market_excess_train'))
    model_capm = sm.OLS(ticker_excess_train, X_capm).fit(cov_type='HAC', cov_kwds={'maxlags': int(np.sqrt(len(ticker_excess_train)))})

   
    #juntar os parâmetros do capm
    params_capm.append({
        'Ticker':ticker,
        'Alpha estimado':model_capm.params.get('const', 0),
        'Beta estimado':model_capm.params.get('Market_excess_train',0),
        'P-Valor Alfa estimado':model_capm.pvalues['const'],
        'P-Valor Beta estimado':model_capm.pvalues['Market_excess_train']
    })
    # Teste de White (Heterocedasticidade geral)
    white_test = het_white(model_capm.resid, model_capm.model.exog)
    white_pvalue = white_test[1]

    # Teste de Durbin-Watson
    dw_stat = durbin_watson(model_capm.resid)

    # EFEITOS ARCH ARCH-LM
    arch_test = het_arch(model_capm.resid.dropna(), nlags=1, store=False)
    arch_pvalue = arch_test[1]


    
    #Juntar os testes de resíduos do CAPM
    tests_capm.append({
        'Ticker':ticker,
        'White':white_pvalue,
        'Durbin-Watson':dw_stat,
        'P-Valor Alfa':model_capm.pvalues['const'],
        'ARCH LM':arch_pvalue,
        'P-Valor Beta':model_capm.pvalues['Market_excess_train']
        
    })

tests_capm_df = pd.DataFrame(tests_capm)
tests_capm_df.to_excel('tests_capm.xlsx', index=False)
params_capm_df = pd.DataFrame(params_capm)
params_capm_df.to_excel('params_capm.xlsx', index=False)

In [ ]:
# Ativar a interface R-Python
pandas2ri.activate()

# Inicializar listas para coletar os resultados
forecast_garchm = []
tests_garchm = []
params_garchm = []

# Configurar o R para usar o idioma inglês e evitar problemas de codificação
ro.r('Sys.setlocale("LC_ALL", "C")')
ro.r('Sys.setenv(LANG = "en")')

# Loop através de cada ticker
for ticker in stock_excess_train['Ticker'].unique():
    ticker_data = stock_excess_train.loc[stock_excess_train['Ticker'] == ticker, 'Stock_Excess']
    market_excess = market_excess_train.loc[ticker_data.index]
    
    # Converter os dados para objetos R
    r_returns = pandas2ri.py2rpy(pd.Series(ticker_data.values, name="returns"))
    r_market_excess = pandas2ri.py2rpy(pd.Series(market_excess.values, name="market"))
    ro.globalenv['returns'] = r_returns
    ro.globalenv['market'] = r_market_excess

    # Código R para ajustar o modelo e extrair os resultados
    r_code = """
    library(rugarch)
    library(lmtest)
    library(FinTS)
    
    # Especificação do modelo GARCH-M com variável exógena
    spec <- ugarchspec(
        mean.model = list(
            armaOrder = c(0, 0),
            include.mean = TRUE,
            archm = TRUE,
            archpow = 1,
            external.regressors = as.matrix(market)
        ),
        variance.model = list(
            model = "sGARCH",
            garchOrder = c(1,1)
        ),
        distribution.model = "norm"
    )
    
    # Tentar ajustar o modelo
    fit <- tryCatch(
        ugarchfit(spec, data = returns),
        error = function(e) { return(NULL) }
    )
    
    # Verificar se o ajuste foi bem-sucedido
    if (is.null(fit)) {
        stop("Falha no ajuste do modelo.")
    }
    
    # Calcular a matriz de covariância robusta
    cov_matrix <- vcov(fit, robust = TRUE)
    params <- coef(fit)
    std_errors <- sqrt(diag(cov_matrix))
    t_values <- params / std_errors
    p_values <- 2 * (1 - pnorm(abs(t_values)))

    # Criar tabela de parâmetros
    params_table <- data.frame(
        Estimate = params,
        Std_Error = std_errors,
        t_value = t_values,
        p_value = p_values
    )
    rownames(params_table) <- names(params)
    
    # Prever a volatilidade condicional
    forecast <- ugarchforecast(fit, n.ahead = 373)
    sigma_forecast <- as.numeric(sigma(forecast))
    
    # Testes de hipótese
    std_residuals <- residuals(fit, standardize=TRUE)
    ljung_box_test <- Box.test(std_residuals, lag=12, type="Ljung-Box", fitdf=0)
    arch_lm_test <- ArchTest(std_residuals, lags=1)
    white_test <- bptest(lm(std_residuals ~ market + I(market^2)))

    # Extrair AIC e BIC como valores numéricos
    AIC_value <- as.numeric(infocriteria(fit)[[1]])
    BIC_value <- as.numeric(infocriteria(fit)[[2]])

    
    # Retornar resultados
    list(
        params_table = params_table,
        sigma_forecast = sigma_forecast,
        ljung_box_p_value = ljung_box_test$p.value,
        arch_lm_p_value = arch_lm_test$p.value,
        white_pvalue = white_test$p.value,
        AIC = AIC_value,
        BIC = BIC_value
    )
    """

    # Executar o código R
    try:
        r_result = ro.r(r_code)
        
        # Verificar se o resultado foi gerado corretamente
        if r_result is None:
            print(f"Erro ao processar o ticker {ticker}: Nenhum resultado gerado.")
            continue
        
        # Extrair os resultados
        params_table = pd.DataFrame(ro.conversion.rpy2py(r_result.rx2('params_table')))
        params_table['Parameter'] = params_table.index
        params_table['Ticker'] = ticker
        params_garchm.append(params_table)

        
        
        sigma_forecast = np.array(r_result.rx2('sigma_forecast'))
        forecast_df = pd.DataFrame({
            'Ticker': ticker,
            'Period': np.arange(1, len(sigma_forecast) + 1),
            'Sigma_Forecast': sigma_forecast
        })
        forecast_garchm.append(forecast_df)
        
        ljung_box_p_value = r_result.rx2('ljung_box_p_value')[0]
        arch_lm_p_value = r_result.rx2('arch_lm_p_value')[0]
        white_pvalue = r_result.rx2('white_pvalue')[0]
        AIC = float(r_result.rx2('AIC')[0])
        BIC = float(r_result.rx2('BIC')[0])
        
        tests_garchm.append({
            'Ticker': ticker,
            'ljung_box_p_value': ljung_box_p_value,
            'arch_lm_p_value': arch_lm_p_value,
            'white_pvalue': white_pvalue,
            'AIC': AIC,
            'BIC': BIC
        })
        
    except Exception as e:
        print(f"Erro ao processar o ticker {ticker}: {e}")
        continue

# Após o loop, salvar os resultados em arquivos Excel
params_garchm_df = pd.concat(params_garchm, ignore_index=True)
params_garchm_df.to_excel('params_garchm.xlsx', index=False)

forecast_garchm_df = pd.concat(forecast_garchm, ignore_index=True)
forecast_garchm_df.to_excel('forecast_garchm.xlsx', index=False)

tests_garchm_df = pd.DataFrame(tests_garchm)
tests_garchm_df.to_excel('tests_garchm.xlsx', index=False)


In [ ]:
# Resetar o índice para ter 'Date' como coluna
#cdi_data_forecast_reset = cdi_data_forecast.reset_index()
# Converter 'CDI_Return' em pontos percentuais
#cdi_data_forecast_reset['CDI_Return_Percent'] = cdi_data_forecast_reset['CDI_Return'] * 100

params_pivot=params_garchm_df.pivot(index='Ticker', columns='Parameter', values='Estimate').reset_index()

market_excess_forecast_garchm_df = pd.DataFrame({
    'Date': market_excess_forecast.index,
    'Market_Excess': market_excess_forecast.values
}).reset_index(drop=True)

market_excess_forecast_garchm_df['Period'] = market_excess_forecast_garchm_df.index + 1  # Period começa em 1

forecast_garchm_df = pd.merge(
    forecast_garchm_df,
    market_excess_forecast_garchm_df[['Period', 'Date', 'Market_Excess']],
    on='Period',
    how='left'
)
forecasts = []

In [ ]:
for ticker in params_pivot['Ticker']:
    # Obter os parâmetros para o ticker atual
    params = params_pivot[params_pivot['Ticker'] == ticker].iloc[0]
    mu = params.get('mu')
    mxreg1 = params.get('mxreg1')
    archm = params.get('archm')
    
    
    # Obter as previsões de sigma para o ticker atual
    sigma_forecast_ticker = forecast_garchm_df[forecast_garchm_df['Ticker'] == ticker].copy()
        
    # Aplicar a fórmula para calcular o retorno em excesso previsto
    sigma_forecast_ticker['Retorno_Excesso_Previsto'] = (
        mu + mxreg1 * sigma_forecast_ticker['Market_Excess'] + archm * sigma_forecast_ticker['Sigma_Forecast']
    )
    
    # Adicionar o ticker e a data ao DataFrame
    sigma_forecast_ticker['Ticker'] = ticker
    sigma_forecast_ticker['Date'] = sigma_forecast_ticker['Date']
    
    # Selecionar as colunas relevantes
    forecasts.append(sigma_forecast_ticker[['Ticker', 'Date', 'Retorno_Excesso_Previsto']])

# Concatenar todos os DataFrames em um único DataFrame
results_garchm_df = pd.concat(forecasts, ignore_index=True)

# Salvar os resultados em um arquivo Excel
results_garchm_df.to_excel('retorno_excesso_previsto.xlsx', index=False)

In [ ]:
# Mesclar com base na coluna 'Date'
#forecasts_with_cdi = pd.merge(
#    results_garchm_df,
 #   cdi_data_forecast_reset[['Date', 'CDI_Return_Percent']],
  #  on='Date',
  #  how='left'
#)

# Calcular o Retorno Total Previsto
#forecasts_with_cdi['Retorno_Total_Previsto'] = (
#    forecasts_with_cdi['Retorno_Excesso_Previsto'] + forecasts_with_cdi['CDI_Return_Percent']
#)

# Passo 4: Salvar os Resultados
#forecasts_with_cdi.to_excel('retorno_total_previsto_garchm.xlsx', index=False)


In [ ]:
retorno_excesso_capm = []

# Criar um DataFrame com as previsões de excesso de mercado
market_excess_forecast_capm_df = pd.DataFrame({
    'Date': market_excess_forecast.index,
    'Market_Excess': market_excess_forecast.values
}).reset_index(drop=True)


# Iterar sobre cada ticker para calcular o Retorno em Excesso Previsto
for index, row in params_capm_df.iterrows():
    ticker = row['Ticker']
    alpha = row['Alpha estimado']
    beta = row['Beta estimado']
        
    # Criar um DataFrame temporário para o ticker atual
    ticker_forecast = pd.DataFrame({
        'Date': market_excess_forecast_capm_df['Date'],
        'Ticker': ticker,
        'Market_Excess': market_excess_forecast_capm_df['Market_Excess']
    })
    
    # Calcular o Retorno em Excesso Previsto
    ticker_forecast['Retorno_Excesso_Previsto'] = alpha + beta * ticker_forecast['Market_Excess']
    
    # Adicionar o DataFrame temporário à lista
    retorno_excesso_capm.append(ticker_forecast[['Ticker', 'Date', 'Retorno_Excesso_Previsto']])

# Concatenar todos os DataFrames em um único DataFrame
retorno_excesso_capm_df = pd.concat(retorno_excesso_capm, ignore_index=True)

# Salvar os retornos em excesso do capm
retorno_excesso_capm_df.to_excel('retorno_excesso_previsto_capm.xlsx', index=False)

In [ ]:
# Mesclar o Retorno em Excesso Previsto com os dados do CDI
#forecasts_capm_with_cdi = pd.merge(
   # retorno_excesso_capm_df,
   # cdi_data_forecast_reset[['Date', 'CDI_Return_Percent']],
  #  on='Date',
 #   how='left'
#)
# Calcular o Retorno Total Previsto
#forecasts_capm_with_cdi['Retorno_Total_Previsto'] = (
 #   forecasts_capm_with_cdi['CDI_Return_Percent'] + forecasts_capm_with_cdi['Retorno_Excesso_Previsto']
#)

# Selecionar as colunas relevantes
#retorno_total_capm_df = forecasts_capm_with_cdi[['Ticker', 'Date', 'Retorno_Total_Previsto']]

# Salvar os resultados em um arquivo Excel
#retorno_total_capm_df.to_excel('retorno_total_previsto_capm.xlsx', index=False)

In [ ]:
#Plotar o comparativo de retornos em excesso
#for ticker in tickers:


 #   plt.figure(figsize=(14, 7))
  #  plt.plot(
   #     market_excess_forecast.index, 
    #    results_garchm_df['Retorno_Excesso_Previsto'][results_garchm_df['Ticker'] == ticker],
     #   label='GARCH-M', 
      #  color='blue'
    #)
    
    # Plotar os retornos em excesso previstos pelo CAPM
    #plt.plot(
     #   market_excess_forecast.index, 
      #  retorno_excesso_capm_df['Retorno_Excesso_Previsto'][retorno_excesso_capm_df['Ticker'] == ticker],
       # label='CAPM', 
       # color='orange'
    #)

    # Plotar os retornos em excesso previstos pelo CAPM
    #plt.plot(
     #   market_excess_forecast.index, 
      #  stock_excess_forecast['Stock_Excess'][stock_excess_forecast['Ticker']==ticker],
      #  label='real', 
      #  color='green'
    #)
    # Adicionar rótulos e título
    #plt.xlabel('Data')
    #plt.ylabel('Retorno em Excesso Previsto (%)')
    #plt.title(f'Comparativo de Retornos em Excesso Previsto - {ticker}')
    
    # Adicionar legenda e grid
    #plt.legend()
    #plt.grid(True)
    
    # Ajustar layout e exibir o gráfico
    #plt.tight_layout()
    #plt.show()
    #plt.close()

In [ ]:
# Definir o nível de significância
significance_level = 0.05

# Filtrar os tickers que não apresentam heterocedasticidade e autocorrelação no modelo GARCH-M
garchm_filtered = tests_garchm_df[
    (tests_garchm_df['white_pvalue'] > significance_level) &
    (tests_garchm_df['ljung_box_p_value'] > significance_level)
]

print(f"\nTickers que passaram os testes GARCH-M ({len(garchm_filtered)} tickers):")
print(garchm_filtered['Ticker'].tolist())

# Filtrar os tickers que não apresentam heterocedasticidade e autocorrelação no modelo CAPM
capm_filtered = tests_capm_df[
    (tests_capm_df['White'] > significance_level) &
    (tests_capm_df['Durbin-Watson'] >= 1.8915) &
    (tests_capm_df['Durbin-Watson'] <= 2.1085)
]

print(f"\nTickers que passaram os testes CAPM ({len(capm_filtered)} tickers):")
print(capm_filtered['Ticker'].tolist())

# Encontrar os tickers que passaram em ambos os modelos
tickers_garchm = set(garchm_filtered['Ticker'])
tickers_capm = set(capm_filtered['Ticker'])

tickers_final = tickers_garchm.intersection(tickers_capm)

# Converter para lista e ordenar
tickers_final = sorted(list(tickers_final))

print(f"\nTickers que não apresentam heterocedasticidade e autocorrelação em ambos os modelos GARCH-M e CAPM ({len(tickers_final)} tickers):")
print(tickers_final)


In [ ]:
#Plotar o comparativo de retornos em excesso apenas dos que não apresentam autocorrelação nem heterocedasticidade para ambos
for ticker in tickers_final:


    plt.figure(figsize=(14, 7))
    plt.plot(
        market_excess_forecast.index, 
        results_garchm_df['Retorno_Excesso_Previsto'][results_garchm_df['Ticker'] == ticker],
        label='GARCH-M', 
        color='blue'
    )
    
    # Plotar os retornos em excesso previstos pelo CAPM
    plt.plot(
        market_excess_forecast.index, 
        retorno_excesso_capm_df['Retorno_Excesso_Previsto'][retorno_excesso_capm_df['Ticker'] == ticker],
        label='CAPM', 
        color='orange'
    )
    
    # Adicionar rótulos e título
    plt.xlabel('Data')
    plt.ylabel('Retorno em Excesso Previsto (%)')
    plt.title(f'Comparativo de Retornos em Excesso Previsto - {ticker}')
    
    # Adicionar legenda e grid
    plt.legend()
    plt.grid(True)
    
    # Ajustar layout e exibir o gráfico
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
#Calcula o RMSE e a significancia do RMSE entre o GARCH-M e o CAPM com os tickers em comum

rmse=[]
for ticker in tickers_final:

    rmse_garchm=root_mean_squared_error(stock_excess_forecast['Stock_Excess'][stock_excess_forecast['Ticker']==ticker],results_garchm_df['Retorno_Excesso_Previsto'][results_garchm_df['Ticker'] == ticker])
    rmse_capm=root_mean_squared_error(stock_excess_forecast['Stock_Excess'][stock_excess_forecast['Ticker']==ticker],retorno_excesso_capm_df['Retorno_Excesso_Previsto'][retorno_excesso_capm_df['Ticker'] == ticker])

    # Calcular os erros quadrados de cada modelo
    errors_garchm = (stock_excess_forecast['Stock_Excess'][stock_excess_forecast['Ticker']==ticker].values - results_garchm_df['Retorno_Excesso_Previsto'][results_garchm_df['Ticker'] == ticker].values) ** 2
    errors_capm = (stock_excess_forecast['Stock_Excess'][stock_excess_forecast['Ticker']==ticker].values - retorno_excesso_capm_df['Retorno_Excesso_Previsto'][retorno_excesso_capm_df['Ticker'] == ticker].values) ** 2

    # Realizar o teste t pareado entre os erros quadrados dos dois modelos
    t_stat, p_value = ttest_rel(errors_garchm, errors_capm)
    
    rmse.append({
        'Ticker': ticker,
        'RMSE - GARCHM': rmse_garchm,
        'RMSE - CAPM': rmse_capm,
        't-statistic': t_stat,
        'p-value': p_value
    })

rmse_df = pd.DataFrame(rmse)
rmse_df.to_excel('RMSE.xlsx', index=False)

In [ ]:
#Calcula o RMSE do GARCH-M para todos os tickers que limparam

rmse_garchm=[]
for ticker in tickers_garchm:

    rmse_g=root_mean_squared_error(stock_excess_forecast['Stock_Excess'][stock_excess_forecast['Ticker']==ticker],results_garchm_df['Retorno_Excesso_Previsto'][results_garchm_df['Ticker'] == ticker])
    
    rmse_garchm.append({
        'Ticker': ticker,
        'RMSE - GARCHM': rmse_g,
    })

rmse_garchm_df = pd.DataFrame(rmse_garchm)
rmse_garchm_df.to_excel('RMSE_GARCHM.xlsx', index=False)

In [ ]:
# Lista para armazenar os resultados
adf_stock_results = []
adf_market_results = []

# Aplicar o teste ADF para o excesso de retorno de mercado
result_market = adfuller(market_excess_train, autolag='AIC')
adf_market_results.append({
    'ADF Statistic': result_market[0],
    'p-value': result_market[1],
    'Lags Used': result_market[2],
    'Number of Observations': result_market[3],
    'Critical Values': result_market[4],
    'Stationary': result_market[1] < 0.05  # True se p-value < 0.05
})

# Aplicar o teste ADF para cada série de ativos
for ticker in tickers:
    # Filtrar os retornos da série do ticker
    ticker_returns = stock_excess_train.loc[stock_excess_train['Ticker'] == ticker, 'Stock_Excess']
    # Executar o teste ADF
    result = adfuller(ticker_returns, autolag='AIC')
    
    # Coletar os resultados
    adf_stock_results.append({
        'Ticker': ticker,
        'ADF Statistic': result[0],
        'p-value': result[1],
        'Lags Used': result[2],
        'Number of Observations': result[3],
        'Critical Values': result[4],
        'Stationary': result[1] < 0.05  # True se p-value < 0.05
    })

# Converter os resultados em DataFrame
adf_stock_results_df = pd.DataFrame(adf_stock_results)
adf_market_results_df = pd.DataFrame(adf_market_results)

# Salvar os resultados em Excel
adf_stock_results_df.to_excel('ADF_Stock.xlsx', index=False)
adf_market_results_df.to_excel('ADF_Market.xlsx', index=False)